## Building a Postgres DB for Scratch

The goal of this guided project is to setup a postgres database from scratch and load the Boston crime csv data into it.

We will create two user groups:

* readonly: Users in this group will have permission to read data only.
* readwrite: Users in this group will have permissions to read and alter data but not to delete tables.


## Creating the Crime Databse

Database name: crime_db 

Schema name: 'crimes' for storing the tables for containing the crime data.

The database crime_db does not exist yet so we connect to dq.

In [1]:
import psycopg2

conn = psycopg2.connect('dbname=dq user=dq')

# autocommit is required in creating a databse
conn.autocommit = True

cur = conn.cursor()

# create crimes db
cur.execute("CREATE DATABASE crime_db;")
conn.close()

In [2]:
# Now that we have created our crime db, w can connect to it

conn = psycopg2.connect('dbname=crime_db user=dq')

conn.autocommit = True

cur = conn.cursor()

# create the crimes schema
cur.execute ("CREATE SCHEMA crimes;")

## Obtaining the Column Names and Sample

Let's start by reading the column names from the boston.csv file as well as the first row. In this way we will have them at hand throughout this guided project so that we can easily take a look at them at any moment.

In [3]:
import csv
with open('boston.csv') as file:
    reader = csv.reader(file)
    
    col_headers = next(reader)
    first_row = next(reader)
    
col_headers

['incident_number',
 'offense_code',
 'description',
 'date',
 'day_of_the_week',
 'lat',
 'long']

In [4]:
first_row

['1',
 '619',
 'LARCENY ALL OTHERS',
 '2018-09-02',
 'Sunday',
 '42.35779134',
 '-71.13937053']

## Creating an Auxiliary Function

Before we create a table for storing the crime data, we need to identify the proper datatypes for the columns. 

To help us with that, let's create a function — get_col_value_set() — that given the name of a CSV file and a column index (starting at 0) that computes a Python set with all distinct values contained in that column. 


This function will be useful for two reasons:

1. Checking whether an enumerated datatype might be a good choice for representing a column.
2. Computing the maximum length of any text-like column to select appropriate sizes for `VARCHAR` columns.


In [5]:
# print number of unique values for all columns

def get_col_set(csv_file, col_index):
    import csv
    values = set()
    with open(csv_file, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values

for i in range(len(col_headers)):
    values = get_col_set("boston.csv", i)
    print(col_headers[i], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


## Finding the Maximum Length

We can use the get_col_set function to compute the number of distinct values for each column. Columns with a low number of distinct values tend to be good candidates for enumerated datatypes. 

Another important aspect is to know the longest word in any column containing textual data.

Let's compute the maximum length of each value in the description column.

In [6]:
# Print the value of col_headers to see which index the description column is.
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [7]:
descriptions = get_col_set('boston.csv', 2)

max_len = 0

for description in descriptions:
    max_len = max(max_len, len(descriptions))

print(max_len)

239


## Creating the Table

We haveto create an enumerated datatype named weekday for the day_of_the_week since there only seven possible values.

For the incident_number we have decided to use the type INTEGER and set it as the primary key. The same datatype was also used to represent the offense_code.

Since the description has at most 58 character we decided to use the datatype VARCHAR(100) for representing it. This leave some margin while not being so big that we will waste a lot of memory.

The date was represented as the DATE datatype. Finally, for the latitude and longitude we used DECIMAL datatypes.

In [8]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [9]:
print(first_row)

['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


Only column day_of_the_week has a small range of values so we will only create an enumerated datatype for this column. Column offense_code is also a good candidate since there is probably a limited set of possible offense codes.

We saw that the offense_code column has size at most 59. To be on the safe side we will limit the size of the description to 100 and use the VARCHAR(100) datatype.

The lat and long column see to need to hold quite a lot of precision so we will use the decimal type.

In [10]:
# create the enumerated datatype for representing the weekday
cur.execute("""
            CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')
""")

In [11]:
# Let's now create our table

cur.execute("""
        CREATE TABLE crimes.boston_crime(
            incident_number INTEGER PRIMARY KEY,
            offense_code INTEGER,
            description VARCHAR(100),
            date DATE,
            day_of_the_week weekday,
            lat decimal,
            long decimal
        );
""")

## Loading the data

Let's use the Wcopy_expert() function to load the data as it is very fast and very succinct to use.

In [12]:
with open('boston.csv') as f:
    cur.copy_expert("COPY crimes.boston_crime FROM STDIN WITH CSV HEADER;", f)
    

# Print the whole dataset

cur.execute("SELECT * FROM crimes.boston_crime")

# print the number of rows to ensure that they were loaded
print(len(cur.fetchall()))

298329


## Revoking Public Privileges

We revoke all privileges of the public group on the public schema to ensure that users will not inherit privileges on that schema such as the ability to create tables in the public schema.

We also need to revoke all privileges in the newly created schema. Doing this also makes it so that we do not need to revoke the privileges when we create users and groups because unless specified otherwise, privileges are not granted by default.

In [13]:
# revoke privileges on newly created schema
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")

# revoke privileges on database
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

## Creating the read only group

We create a readonly group with NOLOGIN because it is a group and not a user. We grant the group the ability to connect to the crime_db and the ability to use the crimes schema.

Then we deal wit tables privileges by granting SELECT. We also add an extra line compared with what was asked. This extra line changes the way that privileges are given by default to the readonly group on new table that are created on the crimes schema. As we mentioned, by default not privileges are given. However we change is so that by default any user in the readonly group can issue select commands.

In [15]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")


## Creating the read-write group

We create a readwrite group with NOLOGIN because it is a group and not a user. We grant the group the ability to connect to the crime_db and the ability to use the crimes schema.

Then we deal wit tables privileges by granting SELECT, INSERT, UPDATE and DELETE. As before we change the default privileges so that user in the readwrite group have these privileges if we ever create a new table on the crimes schema.


In [16]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

## CREATING USERS FOR EACH GROUP

We create a user named data_analyst with password secret1 in the readonly group.

We create a user named data_scientist with password secret2 in the readwrite group.

In [17]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")

cur.execute("GRANT readonly TO data_analyst")

In [18]:
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")

cur.execute("GRANT readwrite TO data_scientist")

## TEST THE DATABASE

Test the database setup using SQL queries on the pg_roles table and information_schema.table_privileges.

In the pg_roles table we will check database related privileges and for that we will look at the following columns:

* rolname: The name of the user / group that the privilege refers to.
* rolsuper: Whether this user / group is a super user. It should be set to False on every user / group that we have created.
* rolcreaterole: Whether user / group can create users, groups or roles. It should be False on every user / group that we have created.
* rolcreatedb: Whether user / group can create databases. It should be False on every user / group that we have created.
* rolcanlogin: Whether user / group can login. It should be True on the users and False on the groups that we have created.

In the information_schema.table_privileges we will check privileges related to SQL queries on tables. We will list the privileges of each group that we have created.

In [19]:
# Close the old connection to test witha new connection
conn.close()

In [21]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()

# Check users and groups

cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")

for user in cur:
    print(user)
print()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)



In [24]:
# Check privileges

cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")

ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



InternalError: current transaction is aborted, commands ignored until end of transaction block
